[![ Click here to deploy.](https://uohmivykqgnnbiouffke.supabase.co/storage/v1/object/public/landingpage/brevdeploynavy.svg)](https://console.brev.dev/launchable/deploy?launchableID=env-2p8Dz9VJ2kFSrTQMXVRtUMgymKx)

<div class="alert alert-block alert-info"> <b>NOTE</b> It takes about 10 minutes to deploy this notebook as a Launchable. As of this writing, we are working on a free tier so a credit card may be required. You can reach out to your NVIDIA rep for credits. </div>

# ESM-2 Inference

This tutorial serves as a demo for [ESM2](https://www.science.org/doi/abs/10.1126/science.ade2574) Inference using a CSV file with `sequences` column. To pre-train the ESM2 model please refer to [ESM-2 Pretraining](./pretrain.md) tutorial.

<div class="alert alert-block alert-info"> <b>NOTE</b> Some of the cells below generate long text output. We're using <pre>%%capture --no-display --no-stderr cell_output</pre> to suppress this output. Comment or delete this line in the cells below to restore full output.</div>

## Setup and Assumptions

In this tutorial, we will demonstrate how to download ESM2 checkpoint, create a CSV file with protein sequences, and infer a ESM-2 model.

All commands should be executed inside the BioNeMo docker container, which has all ESM-2 dependencies pre-installed. For more information on how to build or pull the BioNeMo2 container, refer to the [Initialization Guide](../../getting-started/initialization-guide.md).

### Import Required Libraries

In [1]:
%%capture --no-display --no-stderr cell_output

import os
import torch
import shutil
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')


### Work Directory

Set the work directory to store data and results:

<div class="alert alert-block alert-info"> <b>NOTE</b> We set the following to clean up the work directory created by this notebook  <pre>cleanup : bool = True</pre></div>

In [2]:
cleanup : bool = True

In [3]:
work_dir="/workspace/bionemo2/esm2_inference_tutorial"

if cleanup and os.path.exists(work_dir):
    shutil.rmtree(work_dir)

if not os.path.exists(work_dir):
    os.makedirs(work_dir)
    print(f"Directory '{work_dir}' created.")
else:
    print(f"Directory '{work_dir}' already exists.")

Directory '/workspace/bionemo2/esm2_inference_tutorial' created.


## Download Model Checkpoints

The following code will download the pre-trained model, `esm2n/650m:2.0`, from the NGC registry:

In [4]:
from bionemo.core.data.load import load

checkpoint_path = load("esm2/650m:2.0")
print(checkpoint_path)

/home/bionemo/.cache/bionemo/0798767e843e3d54315aef91934d28ae7d8e93c2849d5fcfbdf5fac242013997-esm2_650M_nemo2.tar.gz.untar


## Data


We use the `InMemoryCSVDataset` class to load the protein sequence data from a `.csv` file. This data file should at least have a `sequences` column and can optionally have a `labels` column used for fine-tuning applications. Here is an example of how to create your own inference input data using a list of sequences in python:

In [5]:
import pandas as pd

artificial_sequence_data = [
    "TLILGWSDKLGSLLNQLAIANESLGGGTIAVMAERDKEDMELDIGKMEFDFKGTSVI",
    "LYSGDHSTQGARFLRDLAENTGRAEYELLSLF",
    "GRFNVWLGGNESKIRQVLKAVKEIGVSPTLFAVYEKN",
    "DELTALGGLLHDIGKPVQRAGLYSGDHSTQGARFLRDLAENTGRAEYELLSLF",
    "KLGSLLNQLAIANESLGGGTIAVMAERDKEDMELDIGKMEFDFKGTSVI",
    "LFGAIGNAISAIHGQSAVEELVDAFVGGARISSAFPYSGDTYYLPKP",
    "LGGLLHDIGKPVQRAGLYSGDHSTQGARFLRDLAENTGRAEYELLSLF",
    "LYSGDHSTQGARFLRDLAENTGRAEYELLSLF",
    "ISAIHGQSAVEELVDAFVGGARISSAFPYSGDTYYLPKP",
    "SGSKASSDSQDANQCCTSCEDNAPATSYCVECSEPLCETCVEAHQRVKYTKDHTVRSTGPAKT",
]

# Create a DataFrame
df = pd.DataFrame(artificial_sequence_data, columns=["sequences"])

# Save the DataFrame to a CSV file
data_path = os.path.join(work_dir, "sequences.csv")
df.to_csv(data_path, index=False)

## Run Inference

Similar to PyTorch Lightning, ESM-2 Inference takes advantage of some key classes:

1. `MegatronStrategy` - To launch and setup parallelism for [NeMo](https://github.com/NVIDIA/NeMo/tree/main) and [Megatron-LM](https://github.com/NVIDIA/Megatron-LM).
2. `Trainer` - To configure training configurations and logging.
3. `ESMFineTuneDataModule` - To load sequence data for both fine-tuning and inference.
4. `ESM2Config` - To configure the ESM-2 model as `BionemoLightningModule`.

Please refer to [ESM-2 Pretraining](./pretrain.md) and [ESM-2 Fine-Tuning](./finetune.md) tutorials for detailed description of these classes.

To run inference on the data created in the previous step, we can use the `infer_esm2` executable which calls `bionemo-framework/sub-packages/bionemo-esm2/src/bionemo/esm2/scripts/infer_esm2.py`. We can get a full description of inference arguments by providing `--help` in the following command:



In [6]:
! infer_esm2 --help

2024-12-16 20:19:23 - faiss.loader - INFO - Loading faiss with AVX512 support.
2024-12-16 20:19:23 - faiss.loader - INFO - Successfully loaded faiss with AVX512 support.
[NeMo W 2024-12-16 20:19:24 nemo_logging:361] /usr/local/lib/python3.10/dist-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
      warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
    
[NeMo W 2024-12-16 20:19:24 nemo_logging:361] /usr/local/lib/python3.10/dist-packages/pyannote/core/notebook.py:134: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
      cm = get_cmap("Set1")
    
usage: infer_esm2 [-h] --checkpoint-path CHECKPOINT_PATH --data-path DATA_PATH
                  --results-path RESULTS_PATH
                  [--precision {fp16,


The hidden states (which are usually the output of each layer in a neural network) can be obtained by using `--include-hiddens` argument when calling the inference function of ESM-2 in BioNeMo Framework.

The hidden states can be converted into fixed-size vector embeddings. This is done by removing the hidden state vectors corresponding to padding tokens, then averaging across the rest. This process is often used when the goal is to create a single vector representation from the hidden states of a model, which can be used for various sequence-level downstream tasks such as classification (e.g. subcellular localization) or regression (e.g. melting temperature prediction). To obtain the embedding results we can use `--include-embeddings` argument.

By passing the hidden state of an amino acid sequence through the BERT language model head, we can obtain output logits at each position and transform them into probabilities. This can happen by using `--include-logits` argument. Logits here are the raw, unnormalized scores that represent the likelihood of each class and are not probabilities themselves; they can be any real number, including negative values.

Now lets call `infer_esm2` executable with relevant arguments to compute and optionally return embeddings, hiddens and logits.

In [7]:
%%capture --no-display --no-stderr cell_output

! infer_esm2 --checkpoint-path {checkpoint_path} \
             --data-path {data_path} \
             --results-path {work_dir} \
             --micro-batch-size 3 \
             --num-gpus 1 \
             --precision "bf16-mixed" \
             --include-hiddens \
             --include-embeddings \
             --include-logits \
             --include-input-ids

## Inference Results

Inference predictions are stored into `.pt` files for each device. Since we only used one device to run the inference (`--num-gpus 1`) in the previous step, the results were written to `{work_dir}/predictions__rank_0.pt` under the work directory of this notebook (defined above). The `.pt` file containes a dictionary of `{'result_key': torch.Tensor}` that be loaded with PyTorch:

In [8]:
import torch
results = torch.load(f"{work_dir}/predictions__rank_0.pt")

for key, val in results.items():
    if val is not None:
        print(f'{key}\t{val.shape}')

token_logits	torch.Size([1024, 10, 128])
hidden_states	torch.Size([10, 1024, 1280])
input_ids	torch.Size([10, 1024])
embeddings	torch.Size([10, 1280])


In this example `data` a python dict with the following keys `['token_logits', 'hidden_states', 'input_ids', 'embeddings']`. Logits (`token_logits`) tensor has a dimension of `[sequence, batch, hidden]` to improve the training performance. We will transpose the first two dimension in the following to have batch-first shape like the rest of the output tensors.

In [9]:
logits = results['token_logits'].transpose(0, 1)  # s, b, h  -> b, s, h
print(logits.shape)

torch.Size([10, 1024, 128])


The last dimension of `token_logits` is 128, with the first 33 positions corresponding to the amino acid vocabulary, followed by 95 paddings. We use the `tokenizer.vocab_size` to filter out the paddings and only keep the 33 vocab positions.

In [10]:
from bionemo.esm2.data.tokenizer import get_tokenizer
tokenizer = get_tokenizer()

tokens = tokenizer.all_tokens
print(f"There are {tokenizer.vocab_size} unique tokens: {tokens}.")

aa_logits = logits[..., :tokenizer.vocab_size]  # filter out the 95 paddings and only keep 33 vocab positions
print(f"Logits shape after removing the paddings in hidden dimension: {aa_logits.shape}")

There are 33 unique tokens: ['<cls>', '<pad>', '<eos>', '<unk>', 'L', 'A', 'G', 'V', 'S', 'E', 'R', 'T', 'I', 'D', 'P', 'K', 'Q', 'N', 'F', 'Y', 'M', 'H', 'W', 'C', 'X', 'B', 'U', 'Z', 'O', '.', '-', '<null_1>', '<mask>'].
Logits shape after removing the paddings in hidden dimension: torch.Size([10, 1024, 33])


Let's set aside the tokens corresponding to the 20 known amino acids.

In [11]:
aa_tokens = ['L', 'A', 'G', 'V', 'S', 'E', 'R', 'T', 'I', 'D', 'P', 'K', 'Q', 'N', 'F', 'Y', 'M', 'H', 'W', 'C']

aa_indices = [i for i, token in enumerate(tokens) if token in aa_tokens]
extra_indices = [i for i, token in enumerate(tokens) if token not in aa_tokens]

The sequence dimension in this example (1024) is representing the max sequence length wich includes paddings, EOS, and BOS. To filter the relevant amino acid information we can use the input sequence IDs in the results to create a mask that can be used to extract the relevant information in `aa_logits`

In [12]:
input_ids = results['input_ids'] # b, s
# mask where non-amino acid tokens are True
mask = torch.isin(input_ids, torch.tensor(extra_indices))

## DDP Inference Support

Although this tutorial is utilizing one devive to run the inference, distributed inference is supported for ESM2 in BioNeMo Framework. One can simply set the the `--num-gpus n` to run distributed inference on `n` devices. The output predictions will be written into `predictions__rank_<0...n-1>.pt` under the `--results-path` provided. Moreover, by optionally including input token IDs with `--include-input-ids` we can snure 1:1 mapping between input sequences and output predictions.

The following snippet can be used to load and collate the predictions into a single dictionary.


```python
import glob
from bionemo.llm.lightning import batch_collator

collated_preditions = batch_collator([torch.load(path) for path in glob.glob(f"{work_dir}/predictions__rank_*.pt")])
for key, val in collated_preditions.items():
    if val is not None:
        print(f'{key}\t{val.shape}')

# token_logits	torch.Size([1024, 10, 128])
# hidden_states	torch.Size([10, 1024, 1280])
# input_ids     torch.Size([10, 1024])
# embeddings	torch.Size([10, 1280])
```

For more in-depth example of inference and converting logits to probabilities please refer to [ESM-2 Mutant Design Tutorial](./mutant-design.ipynb)